https://www.kaggle.com/c/mercedes-benz-greener-manufacturing

## 라이브러리를 이용하여 머신머닝 모델 만들기

* "바퀴를 새로 발명하지 마라."라는 유명한 말이 있습니다.
* 새로운 데이터를 다룰 때마다 머신러닝 모델을 처음부터 만들어야 한다면 매우 힘이 들고 오랜 시간이 걸릴 것입니다.
* 대신 우리는 다른 개발자들이 개발하여 공개한 Python 라이브러리를 이용하여 머신러닝 모델을 만들고, 값을 예측해보겠습니다.

## 목표

* 머신러닝 모델을 생성하여 학습하고 예측 데이터를 제출한다.
* 선형회귀 모델을 사용해본다.

## 라이브러리 로드

In [1]:
# numpy는 수학적 계산과 관련된 작업을 돕는 라이브러리입니다.
# pandas는 큰 데이터를 다룰 수 있도록 돕는 라이브러리입니다.
import numpy as np
import pandas as pd

# matplotlib.pyplot은 데이터와 관련된 그래프를 그릴 수 있도록 돕는 라이브러리입니다.
# seaborn은 데이터와 관련된 그래프를 그릴 수 있도록 돕는 라이브러리입니다.
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn.preprocessing은 데이터 전처리를 돕는 라이브러리입니다.
from sklearn import preprocessing

# os는 운영체제와 관련된 작업을 돕는 라이브러리입니다.
import os

## 라이브러리 설명
### train_test_split
* train_test_split은 학습용 데이터와 테스트 데이터를 나누는 데 사용되는 메소드입니다.
* 학습용 데이터와 테스트 데이터를 분리하는 것에 대해서는 밑에서 구체적으로 학습하겠습니다.
*  https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

### linear_model
* linear_model은 선형회귀모델을 구현한 sklearn의 라이브러리입니다.
*  https://scikit-learn.org/stable/modules/linear_model.html

## 데이터 확인

* 지난번에 데이터 탐색과 전처리를 진행했던 데이터를 불러오고 확인하겠습니다.

In [2]:
base_path = "data/benz"

In [3]:
# train 변수에 train.csv 데이터를 불러옵니다.
# test 변수에 test.csv 데이터를 불러옵니다.
# sub 변수에 sample_submission.csv 데이터를 불러옵니다.

train = pd.read_csv(f'{base_path}/train_preprocessed.csv')
test = pd.read_csv(f'{base_path}/test_preprocessed.csv')
sub = pd.read_csv(f'{base_path}/sample_submission.csv.zip')

In [4]:
# train을 확인합니다.
train

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,32,23,17,0,3,24,9,14,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,32,21,19,4,3,28,11,14,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,20,24,34,2,3,27,9,23,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,20,21,34,5,3,27,11,4,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,20,23,34,5,3,12,3,13,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4203,107.39,8,20,16,2,3,0,3,16,0,...,1,0,0,0,0,0,0,0,0,0
4204,108.77,31,16,40,3,3,0,7,7,0,...,0,1,0,0,0,0,0,0,0,0
4205,109.22,8,23,38,0,3,0,6,4,0,...,0,0,1,0,0,0,0,0,0,0
4206,87.48,9,19,25,5,3,0,11,20,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# test를 확인합니다.
test

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,21,23,34,5,3,26,0,22,0,0,...,0,0,0,1,0,0,0,0,0,0
1,42,3,8,0,3,9,6,24,0,0,...,0,0,1,0,0,0,0,0,0,0
2,21,23,17,5,3,0,9,9,0,0,...,0,0,0,1,0,0,0,0,0,0
3,21,13,34,5,3,31,11,13,0,0,...,0,0,0,1,0,0,0,0,0,0
4,45,20,17,2,3,30,8,12,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,6,9,17,5,3,1,9,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4205,42,1,8,3,3,1,9,24,0,0,...,0,1,0,0,0,0,0,0,0,0
4206,47,23,17,5,3,1,3,22,0,0,...,0,0,0,0,0,0,0,0,0,0
4207,7,23,17,0,3,1,2,16,0,0,...,0,0,1,0,0,0,0,0,0,0


In [6]:
# sub를 확인합니다.
sub

,ID,y
0,1,100.669318
1,2,100.669318
2,3,100.669318
3,4,100.669318
4,5,100.669318
...,...,...
4204,8410,100.669318
4205,8411,100.669318
4206,8413,100.669318
4207,8414,100.669318


## Label 이름 변경

* 머신러닝 모델을 만들 때, 예측하려는 Label 데이터를 제외한 데이터를 X, Label 데이터를 y로 정의합니다.
* 변수명은 자유롭게 설정해도 되나, 여기서는 혼동을 막기 위해 'y' 변수명을 'target'으로 변경하겠습니다.

In [7]:
# train의 'y'행을 'target'으로 재명명(rename)합니다.
# train을 확인합니다.
train = train.rename(columns={'y':'target'})
train

,target,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,32,23,17,0,3,24,9,14,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,32,21,19,4,3,28,11,14,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,20,24,34,2,3,27,9,23,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,20,21,34,5,3,27,11,4,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,20,23,34,5,3,12,3,13,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4203,107.39,8,20,16,2,3,0,3,16,0,...,1,0,0,0,0,0,0,0,0,0
4204,108.77,31,16,40,3,3,0,7,7,0,...,0,1,0,0,0,0,0,0,0,0
4205,109.22,8,23,38,0,3,0,6,4,0,...,0,0,1,0,0,0,0,0,0,0
4206,87.48,9,19,25,5,3,0,11,20,0,...,0,0,0,0,0,0,0,0,0,0


## 학습용 데이터와 테스트용 데이터 나누기
* 머신러닝 모델을 만들 때에는 꼭 학습용 데이터와 테스트용 데이터를 분리해야 합니다.
* 이는 모델이 제대로 학습하고 있는지 확인하고 재조정하기 위해서입니다.
<br><br>
* 만약 학습용 데이터에 테스트용 데이터가 섞여 들어간다면, 머신러닝 모델은 검증할 때 학습한 데이터를 그대로 반환할 것입니다.
* 이 경우 머신러닝 모델의 성능은 항샹되지 않았는데 점수는 좋게 나올 것입니다.
* 마치 답지를 가지고 시험을 보는 것과 같은 상황입니다.
* 이런 상황을 데이터 유출(Data Leakage)라고 합니다.
<br><br>
* 데이터 유출이 왜 위험할까요?
* 데이터 유출을 방지하기 위해서는 어떻게 해야 할까요?

## random_state
* 모델이 무작위성을 가지고 있을 경우 실행할 때마다 결과가 달라집니다.
* random_state 파라미터는 난수를 고정시켜 매 실행마다 결과가 같아지게 합니다.
* 이번 예제에서는 random_state = 0으로 통일하겠습니다.

In [8]:
# 변수 X에 'target' 값을 제외한 나머지 변수를 넣습니다.
# 변수 y에 'target' 값을 넣습니다.
X = train.drop(columns=['target'])
y = train['target']

In [9]:
# sklearn.model_selection의 train_test_split은 학습 데이터와 검증 데이터를 나눠주는 함수입니다.
from sklearn.model_selection import train_test_split

# 이 때, 학습할 데이터와 테스트할 데이터의 값의 비율은 9대 1로, random_state는 0으로 하겠습니다.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.9, random_state=0)

In [10]:
# X_train을 확인합니다.
X_train

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
1990,6,7,3,2,3,16,3,24,0,0,...,0,0,1,0,0,0,0,0,0,0
4194,36,13,3,5,3,0,6,9,0,0,...,0,0,0,0,0,0,0,0,0,0
3011,8,13,16,5,3,21,9,16,0,0,...,0,0,0,0,0,0,0,0,0,0
2801,25,1,7,2,3,20,11,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2792,39,3,8,0,3,20,6,21,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,5,22,16,3,3,6,3,18,0,0,...,1,0,0,0,0,0,0,0,0,0
3264,19,1,33,2,3,23,9,20,0,0,...,1,0,0,0,0,0,0,0,0,0
1653,9,19,26,5,3,3,9,23,0,0,...,0,0,0,0,0,0,0,0,0,0
2607,8,20,16,6,3,17,0,5,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
# y_train을 확인합니다.
y_train

1990     94.35
4194     86.23
3011    112.75
2801     97.95
2792     90.14
         ...  
1033    115.81
3264     99.95
1653     87.86
2607    112.04
2732    128.58
Name: target, Length: 3787, dtype: float64

In [12]:
# X_valid를 확인합니다.
X_valid

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
3430,7,13,16,5,3,22,3,21,0,0,...,0,0,0,0,0,0,0,0,0,0
2131,42,22,16,2,3,16,9,24,0,0,...,1,0,0,0,0,0,0,0,0,0
2680,9,19,29,5,3,17,11,15,0,0,...,0,0,0,0,0,0,0,0,0,0
195,45,19,38,5,3,13,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
3032,43,20,16,2,3,21,9,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2616,26,20,3,6,3,17,11,22,0,0,...,1,0,0,0,0,0,0,0,0,0
1330,17,23,8,2,3,2,6,24,0,1,...,1,0,0,0,0,0,0,0,0,0
1009,42,20,16,2,3,6,3,5,0,0,...,1,0,0,0,0,0,0,0,0,0
1953,35,20,3,2,3,7,3,14,0,0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
# y_valid를 확인합니다.
y_valid

3430     96.49
2131     97.58
2680     90.60
195      88.10
3032    106.04
         ...  
2616     93.42
1330    101.74
1009    105.43
1953     91.08
2348     87.80
Name: target, Length: 421, dtype: float64

## 선형회귀 모델

### 선형회귀란?
* 통계학에서, 선형 회귀(線型回歸, 영어: linear regression)는 종속 변수 y와 한 개 이상의 독립 변수 (또는 설명 변수) X와의 선형 상관 관계를 모델링하는 회귀분석 기법이다. 

* https://ko.wikipedia.org/wiki/%EC%84%A0%ED%98%95_%ED%9A%8C%EA%B7%80

In [14]:
# sklearn의 linear_model은 선형회귀 모델을 구현한 라이브러리입니다.
from sklearn import linear_model

# 선형회귀 모델 model_lr을 생성합니다.
model_lr = linear_model.LinearRegression()

In [15]:
# model_lr을 X_train과 y_train으로 학습시킵니다.
model_lr.fit(X_train, y_train)

LinearRegression()

In [16]:
# 머신러닝 모델명을 출력합니다.
# 학습용 세트 정확도(R2 score)를 출력합니다.
# 검증 세트 정확도(R2 score)를 출력합니다.
print("모델 : {}".format(model_lr))
print("학습용 세트 정확도: {:.3f}".format(model_lr.score(X_train, y_train)))
print("검증 세트 정확도: {:.3f}".format(model_lr.score(X_valid, y_valid)))

모델 : LinearRegression()
학습용 세트 정확도: 0.626
검증 세트 정확도: -10545514456898996.000


* 값이 매우 좋지 않습니다.
* 선형 회귀 모델에서 왜 이런 일이 일어날까요?

* 이런 값이 나오는 걸 방지하려면, 어떤 방법을 쓸 수 있을까요?

In [17]:
# 모델의 예측값을 prediction_lr에 담습니다.
# prediction_lr을 확인합니다.
prediction_lr = model_lr.predict(test)
prediction_lr

array([ 1.02545913e+10, -1.90181172e+09,  1.08532425e+02, ...,
        9.29494171e+01,  1.10975601e+02,  9.48889008e+01])

In [18]:
# sub의 'y' 변수에 prediction_lr값을 넣습니다.
# sub를 확인합니다.
sub['y'] = prediction_lr
sub

,ID,y
0,1,1.025459e+10
1,2,-1.901812e+09
2,3,1.085324e+02
3,4,7.725897e+01
4,5,1.111867e+02
...,...,...
4204,8410,1.037022e+02
4205,8411,9.714619e+01
4206,8413,9.294942e+01
4207,8414,1.109756e+02


In [19]:
# sub를 csv 파일로 내보냅니다.
sub.to_csv(f'{base_path}/sub_lr.csv', encoding='utf-8', index=False)

* 점수에는 public score와 private score가 있습니다.
* 대회마다 public score와 private score를 매기는 방식이 다 다르지만, 일반적으로 public score는 전체 데이터 중 일부만을 샘플링하여 채점하고, private score는 전체 데이터 중 더 많은 데이터를 대상으로 채점합니다.
* 참가자들이 얻은 public score는 리더보드에 공개되며 이를 바탕으로 현재 위치를 가늠할 수 있습니다.
* 그러나 최종 결과는 private score로 결정됩니다.


https://www.kaggle.com/c/mercedes-benz-greener-manufacturing